In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
from sklearn.metrics import *

In [9]:
import torch
from sahi import AutoDetectionModel
from sahi.utils.cv import read_image
from sahi.predict import get_sliced_prediction, get_prediction
from sahi.annotation import Category

import os
from fastapi import FastAPI, Query
from typing import List

from unidecode import unidecode

import ftplib
from PIL import Image
from io import BytesIO
from glob import glob

import numpy as np
import pandas as pd
from haversine import haversine
from sklearn.neighbors import BallTree
from tqdm import tqdm

In [4]:
detection_model = AutoDetectionModel.from_pretrained(
    model_type='yolov8',
    model_path='best.pt',
    confidence_threshold=0.5,
    device='cuda:0' if torch.cuda.is_available() else 'cpu'
)

bird_model = AutoDetectionModel.from_pretrained(
    model_type='yolov8',
    model_path='bird_best.pt',
    confidence_threshold=0.5,
    device='cuda:0' if torch.cuda.is_available() else 'cpu'
)

df = pd.read_csv('df_coordinates.csv')
aero = pd.read_csv('apinfo.ru.csv', sep='|', encoding='cp1251')
X = aero[['latitude', 'longitude']].values
tree = BallTree(X, leaf_size=2)

In [5]:
for col in df.columns[4:-2]:
    df.loc[df[col] == 'да', col] = 1
    df.loc[df[col] == 'нда', col] = 1
    df.loc[df[col] == 'нет', col] = 0
    df.loc[df[col] == 'нн', col] = 0
    df.loc[df[col] == 'нет ', col] = 0
    df[col] = df[col].astype(int)

In [6]:
gt = df[[
    'Отсутствие КПП, ворот или шлагбаума ',
    'Отсутствие ванн для дезинфекции колес ',
    'Отсутствие весового контроля ',
    'Не выполняется пересыпка тела полигона',
    'Отсутствие ограждения по периметру полигона ',
    'Отсутствие обводных каналов ',
    'Заболачивание и подтопление полигона ',
    'Выход фильтрата за границы отведенного земельного участка',
    'Наличие тлений и возгораний на теле полигона',
    'Разлет фракций на прилегающие территории ',
    'Размещение фракций, не относящихся к ТКО'
]].values

In [53]:
def proc(paths, obj_id):

    # список для хранения результатов обработки каждого изображения
    result = {
            'КПП': 1,
            'Дезинфекция': 1, 
            'Вес контроль': 1, 
            'ТКО': 1,
            'Ограждение': 1, 
            'Обв канал': 1, 
            'Заболачивание': 0, 
            'Фильтрат': 0, 
            'Тление': 0, 
            'Фракции': 0, 
            'НеТКО': 0,
        }
    
    # проходимся по всем файлам в папке
    for filepath in paths:
        
        # вызываем функцию обработки изображения
        img = read_image(filepath)
        
        pred = get_sliced_prediction(
            img,
            detection_model,
            slice_height=img.shape[0]//3,
            slice_width=img.shape[0]//3,
            overlap_height_ratio=0.2,
            overlap_width_ratio=0.2
        )
            
        # добавляем результат обработки в список
        for obj_pred in pred.object_prediction_list:
            
            if obj_pred.category.name in ['КПП', 'Дезинфекция', 'Вес контроль', 'ТКО', 'Ограждение', 'Обв канал']:
                result[obj_pred.category.name] = 0
            else:
                result[obj_pred.category.name] = 1
                
    return result.values()

In [52]:
gt[:5]

array([[0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1],
       [0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1],
       [1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1],
       [0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0],
       [1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0]])

In [58]:
f1

[0.7837837837837837,
 0.7945205479452054,
 0.8055555555555555,
 0.8055555555555555,
 0.8055555555555555,
 0.7887323943661971,
 0.7887323943661971,
 0.7714285714285715]

In [57]:
for th in range(50, 72, 3):
    print(th)

50
53
56
59
62
65
68
71


In [56]:
f1 = []
confm = []
reps = []

for th in range(50, 72, 3):

    detection_model = AutoDetectionModel.from_pretrained(
        model_type='yolov8',
        model_path='best.pt',
        confidence_threshold=th/100,
        device='cuda:0' if torch.cuda.is_available() else 'cpu'
    )
    
    reses = []
    
    for obj_id in tqdm(df['ID Объекта'].iloc[:5]):
        
        paths = glob(f'images/{obj_id}*')[:2]
        
        res = proc(paths, obj_id)
        
        reses.append(list(res))
        
    print(classification_report(gt[:5].flatten(), np.asarray(reses).flatten()))
    print(confusion_matrix(gt[:5].flatten(), np.asarray(reses).flatten()))
    
    
    f1.append(f1_score(gt[:5].flatten(), np.asarray(reses).flatten()))
    confm.append(confusion_matrix(gt[:5].flatten(), np.asarray(reses).flatten()))
    reps.append(classification_report(gt[:5].flatten(), np.asarray(reses).flatten()))

  0%|                                                                                            | 0/5 [00:00<?, ?it/s]

Performing prediction on 32 number of slices.


  0%|                                                                                            | 0/5 [00:03<?, ?it/s]

Performing prediction on 32 number of slices.


 20%|████████████████▊                                                                   | 1/5 [00:07<00:27,  6.78s/it]

...

Performing prediction on 32 number of slices.


 80%|███████████████████████████████████████████████████████████████████▏                | 4/5 [00:31<00:06,  6.97s/it]

Performing prediction on 32 number of slices.


100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:34<00:00,  6.91s/it]

              precision    recall  f1-score   support

           0       0.63      0.57      0.60        21
           1       0.75      0.79      0.77        34

    accuracy                           0.71        55
   macro avg       0.69      0.68      0.69        55
weighted avg       0.70      0.71      0.71        55

[[12  9]
 [ 7 27]]


In [ ]:
 precision    recall  f1-score   support

           0       0.71      0.57      0.63        21
           1       0.76      0.85      0.81        34

    accuracy                           0.75        55
   macro avg       0.73      0.71      0.72        55
weighted avg       0.74      0.75      0.74        55

[[12  9]
 [ 5 29]]

In [59]:
result = {
            'image_paths': [],
            'КПП': 1,
            'Дезинфекция': 1, 
            'Вес контроль': 1, 
            'ТКО': 1,
            'Ограждение': 1, 
            'Обв канал': 1, 
            'Заболачивание': 0, 
            'Фильтрат': 0, 
            'Тление': 0, 
            'Фракции': 0, 
            'НеТКО': 0,
            'Птицы': 0,
            'Аэропорт': 0
        }

In [69]:
len(['КПП','Дезинфекция','Вес контроль','ТКО','Ограждение','Обв канал','Заболачивание','Фильтрат','Тление','Фракции','НеТКО','Птицы','Аэропорт'])

13

In [71]:
len(['Отсутствие КПП, ворот или шлагбаума',
'Отсутствие ванн для дезинфекции колес',
'Отсутствие весового контроля',
'Не выполняется пересыпка тела полигона',
'Отсутствие ограждения по периметру полигона',
'Отсутствие обводных каналов',
'Заболачивание и подтопление полигона',
'Выход фильтрата за границы отведенного земельного участка',
'Наличие тлений и возгораний на теле полигона',
'Разлет фракций на прилегающие территории',
'Размещение фракций, не относящихся к ТКО',
'Наличие птиц на участке',
'Расположение объекта на расстоянии менее 15 км от действующего аэродрома'])

13

In [68]:
num_result

[0, 1, 2, 3, 4, 5]